## 1. Libraries and settings

In [ ]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt

from datetime import date

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter("ignore", UserWarning)


# #hparams

# # Update these dimensions based on your dataset

hidden_dim = 32
num_layers = 2

num_epochs = 5
learning_rate=0.01
step_size=30
gamma=0.9

dropout_rate=0.2


In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

df['signal'] = df['signal'].replace({'SignalNone': 1, 'SignalLong': 2, 'SignalShort': 0})
df.fillna(method='pad');


In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()

# predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()
predictor_list = [('cdlmorningdojistar', 'integer'), 
               ('cdlidentical3crows', 'integer'), 
               ('cdlhangingman', 'integer')]


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)


In [ ]:
from sklearn.model_selection import train_test_split

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)  # 0.2 here means 20% of the original data, or 25% of the training+validation set


In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a scaler instance
scaler = StandardScaler()

# Fit the scaler to your data and transform
X_train_scaled = scaler.fit_transform(X_train)

# Create a DataFrame from the scaled data with the same index and columns
X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)


# Fit the scaler to your data and transform
X_test_scaled = scaler.fit_transform(X_test)

# Create a DataFrame from the scaled data with the same index and columns
X_test_scaled_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

In [ ]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Assuming y_train is your target labels tensor for the training data
# and it's already in the form of a 1D tensor of class indices (0 to C-1)

# Convert y_train to a numpy array if it's a tensor
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = y_train  # Assuming y_train is already a numpy array

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_np), y=y_train_np)

# Convert class weights to a tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'

In [ ]:
timestep = 80

def create_sequences(input_data, sequence_length):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - sequence_length):
        seq = input_data[i:(i + sequence_length)]
        sequences.append(seq)
    return np.array(sequences)

# Assuming X_train_scaled_df and X_test_scaled_df are already scaled and are DataFrames
X_train_list = create_sequences(X_train_scaled_df.values, timestep)
X_test_list = create_sequences(X_test_scaled_df.values, timestep)


y_train_list = create_sequences(y_train, timestep)
y_test_list = create_sequences(y_test, timestep)



In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert your numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_list, dtype=torch.float)
y_train_tensor = torch.tensor(y_train_list, dtype=torch.long)  # Use torch.long for classification labels

X_test_tensor = torch.tensor(X_test_list, dtype=torch.float)
y_test_tensor = torch.tensor(y_test_list, dtype=torch.long)
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Create DataLoaders
batch_size = 20  # You can adjust the batch size
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)  # Typically no need to shuffle the test set

In [ ]:
print('x_train.shape = ',X_train_tensor.shape)
print('x_test.shape = ',X_test_tensor.shape)
print('y_train.shape = ',y_train_tensor.shape)
print('y_test.shape = ',y_test_tensor.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


input_dim = X_train_list.shape[2]  # Number of features
output_dim = 3  # Number of classes

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        # Adjusted to process the LSTM output at each timestep
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Still doubling hidden_dim for bidirectional output
        
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out)
        
        # Apply the fully connected layer to each timestep
        # No need to concatenate the last hidden states from both directions
        out = self.fc(out)
        
        return out







In [ ]:
num_epochs = 100
print_epochs = 1

# Create the model with bidirectional LSTM
model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim, dropout_rate=dropout_rate)
model.to(device)  # Move your model to the MPS device

# Loss function, optimizer, and scheduler remain the same
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=step_size, gamma=gamma)

# hist will track the loss for now
hist = np.zeros(num_epochs)

# Ensure your model is in training mode
model.train()

for epoch in range(num_epochs):
    batch_losses = []  # List to store batch losses
    
    # Training phase
    for X_batch, y_batch in train_loader:
        # Move X_batch and y_batch to the correct device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        y_train_pred = model(X_batch)
        
        # Compute loss
        loss = 0
        for timestep in range(y_train_pred.shape[1]):  # Iterate through each timestep
            loss += loss_fn(y_train_pred[:, timestep], y_batch[:, timestep])
        loss /= y_train_pred.shape[1]  # Average loss across timesteps
        
        # Backward pass and optimize
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        batch_losses.append(loss.item())
    
    scheduler.step()
    
    # Calculate average loss for the epoch
    epoch_loss = np.mean(batch_losses)
    hist[epoch] = epoch_loss
    
    if epoch % print_epochs == 0:  # Adjust logging frequency according to your preference
        print(f"Epoch {epoch}, Loss: {epoch_loss}")
        
        # Optional: Add accuracy calculation or other metrics here
        # Note: You'd typically calculate validation metrics here using a separate validation set


In [ ]:
# # Adjust the figure size
# plt.figure(figsize=(6, 3))

# # Plot the training loss
# plt.plot(hist, label="Training loss")
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
model.eval()


In [ ]:
X_test_tensor_gpu = X_test_tensor.to(device)


In [ ]:
with torch.no_grad():
    y_test_pred = model(X_test_tensor_gpu)
    
    # Convert logits to probabilities for each class
    probabilities = torch.softmax(y_test_pred, dim=2)  # Assuming the model outputs logits with shape [batch_size, sequence_length, num_classes]

    # Get the predicted class labels for each time step
    _, predicted_labels = torch.max(probabilities, dim=2)

    # Move the predictions back to CPU if needed, and convert to numpy for further processing or evaluation
    predicted_labels_numpy = predicted_labels.cpu().numpy()

In [64]:
predicted_labels_flat = predicted_labels_numpy.flatten()
predicted_labels_flat.shape

(411520,)

In [ ]:
df_split = data.data['symbol'][-len(predicted_labels_numpy):].copy()
df_split.loc[:, "signal"] = predicted_labels_numpy


# signal = df_split['signal']
# entries = signal == 2
# exits = signal == 0

In [ ]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [ ]:
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 600
vbt.settings['plotting']['layout']['height'] = 300

In [ ]:
pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()

In [ ]:
pf.stats()

In [ ]:
stats = pf.stats()
total_return = stats['Total Return [%]']
orders = stats['Total Orders']
print("Total Orders:", orders)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Convert tensors to numpy arrays for use with Scikit-Learn
true_labels = y_test.cpu().numpy()
pred_labels = predicted_labels.cpu().numpy()

precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for unweighted mean
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
conf_matrix = confusion_matrix(true_labels, pred_labels)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:\n', conf_matrix)